In [2]:
import pandas as pd
from scipy.stats import wasserstein_distance
from statistics import mean

### Lettura dati file degli score

In [4]:
PATH_FILE_SCORE = './tabellaScoreNuova.csv'

scores = pd.read_csv(PATH_FILE_SCORE)
scores.head(50)


,Utente1,Utente2,ScoreML
0,12_M_1,12_M_2,0.543131
1,12_M_1,12_M_3,0.507192
2,12_M_1,12_M_4,0.256462
3,12_M_1,12_M_5,0.293741
4,12_M_1,13_M_1,0.268435
5,12_M_1,13_M_2,0.185068
6,12_M_1,13_M_3,0.221212
7,12_M_1,13_M_4,0.232198
8,12_M_1,13_M_5,0.191751
9,12_M_1,15_M_1,0.202532


### Aggiunta colonne di utilità

In [3]:
def calcola_id_soggetto_da_utente(u):
    return f'{u.split("_")[0]}{u.split("_")[1]}'

def calcola_num_acquisizione_da_utente(u):
    return u.split('_')[-1]

In [4]:
scores['Primo soggetto'] = scores['Utente1'].apply(lambda u: calcola_id_soggetto_da_utente(u))
scores['Secondo soggetto'] = scores['Utente2'].apply(lambda u: calcola_id_soggetto_da_utente(u))
scores['Numero prima acquisizione'] = scores['Utente1'].apply(lambda u: calcola_num_acquisizione_da_utente(u))
scores['Numero seconda acquisizione'] = scores['Utente2'].apply(lambda u: calcola_num_acquisizione_da_utente(u))

scores.head()

,Utente1,Utente2,ScoreML,Primo soggetto,Secondo soggetto,Numero prima acquisizione,Numero seconda acquisizione
0,12_M_1,12_M_2,0.543131,12M,12M,1,2
1,12_M_1,12_M_3,0.507192,12M,12M,1,3
2,12_M_1,12_M_4,0.256462,12M,12M,1,4
3,12_M_1,12_M_5,0.293741,12M,12M,1,5
4,12_M_1,13_M_1,0.268435,12M,13M,1,1


### Calcolo delle pseudo-labels

In [5]:
def get_set_intra_classe(utente, scores):
    id_soggetto = calcola_id_soggetto_da_utente(utente)
    id_acquisizione = calcola_num_acquisizione_da_utente(utente)
    # Filtro per soggetto
    soggetto_scores = scores[(scores['Primo soggetto'] == id_soggetto) & (scores['Secondo soggetto'] == id_soggetto)]
    # Filtro per acquisizione
    soggetto_scores = soggetto_scores[(soggetto_scores['Numero prima acquisizione'] == id_acquisizione) | (soggetto_scores['Numero seconda acquisizione'] == id_acquisizione)]
    return set(soggetto_scores["ScoreML"])


def get_set_inter_classe(utente, scores):
    id_soggetto = calcola_id_soggetto_da_utente(utente)
    id_acquisizione = calcola_num_acquisizione_da_utente(utente)    
    
    id_soggetto = calcola_id_soggetto_da_utente(utente)
    id_acquisizione = calcola_num_acquisizione_da_utente(utente)
    # Filtro per soggetto
    soggetto_scores = scores[((scores['Primo soggetto'] == id_soggetto) & (scores['Secondo soggetto'] != id_soggetto)) | ((scores['Primo soggetto'] != id_soggetto) & (scores['Secondo soggetto'] == id_soggetto))]
    # Filtro per acquisizione
    soggetto_scores = soggetto_scores[(soggetto_scores['Utente1'] == utente) | (soggetto_scores['Utente2'] == utente)]
    return set(soggetto_scores["ScoreML"])
    

In [6]:
TRADE_OFF_PARAMETER = 1

lista_utenti = list(set(scores.Utente1.unique()) | set(scores.Utente2.unique()))
lista_utenti.sort()

In [7]:
df_labels = pd.DataFrame(columns=['Acquisizione', 'Label']) 

for utente in lista_utenti:
    set_intra_classe = get_set_intra_classe(utente, scores)
    QxP = mean(set_intra_classe)

    set_inter_classe = get_set_inter_classe(utente, scores)
    list_inter_classe = list(set_inter_classe)
    list_inter_classe.sort(reverse=True)
    list_confronto_intra_classe = list_inter_classe[0: len(set_intra_classe)]
    QxN = wasserstein_distance(list(set_intra_classe), list_confronto_intra_classe)

    label = QxP + TRADE_OFF_PARAMETER * QxN

    row_acquisizione = pd.DataFrame.from_dict({'Acquisizione': utente, 'Label': label}, orient='index').T
    df_labels = pd.concat([df_labels, row_acquisizione])


### Normalizzazione delle labels tra 0 e 100

In [8]:
min_label = df_labels.Label.min()
max_label = df_labels.Label.max()
df_labels['Label_norm'] = df_labels['Label'].apply(lambda l: 100 * (l - min_label)/(max_label - min_label))
df_labels = df_labels.reset_index(drop=True)

In [9]:
df_labels

,Acquisizione,Label,Label_norm
0,12_M_1,0.558931,14.387763
1,12_M_2,0.848155,90.057848
2,12_M_3,0.886156,100.000000
3,12_M_4,0.771945,70.119024
4,12_M_5,0.744003,62.808388
5,13_M_1,0.503938,0.000000
6,13_M_2,0.793221,75.685325
7,13_M_3,0.70952,53.786567
8,13_M_4,0.786351,73.888038
9,13_M_5,0.771363,69.966720


### Analisi su singola acquisizione

In [11]:
acquisizioneAnalisi = '13_M_1'
scores[(scores.Utente1 == acquisizioneAnalisi) | (scores.Utente2 == acquisizioneAnalisi)]

,Utente1,Utente2,ScoreML,Primo soggetto,Secondo soggetto,Numero prima acquisizione,Numero seconda acquisizione
4,12_M_1,13_M_1,0.268435,12M,13M,1,1
32,12_M_2,13_M_1,0.237045,12M,13M,2,1
59,12_M_3,13_M_1,0.216841,12M,13M,3,1
85,12_M_4,13_M_1,0.207680,12M,13M,4,1
110,12_M_5,13_M_1,0.199786,12M,13M,5,1
135,13_M_1,13_M_2,0.444066,13M,13M,1,2
136,13_M_1,13_M_3,0.343519,13M,13M,1,3
137,13_M_1,13_M_4,0.407245,13M,13M,1,4
138,13_M_1,13_M_5,0.325882,13M,13M,1,5
139,13_M_1,15_M_1,0.166957,13M,15M,1,1
